In [ ]:
import sys
import math
sys.path.append('../../python/')  
from periphery import logicGate
from periphery import constant
print(constant.INV)

In [ ]:


MIN_NMOS_SIZE = 1.0
MAX_TRANSISTOR_HEIGHT = 2.5


class SRAMWriteDriver:
    def __init__(self, input_param, tech, cell):
        self.input_param = input_param
        self.tech = tech
        self.cell = cell
        self.initialized = False

    def initialize(self, num_col, activity_col_write, num_write_cell_per_op):
        self.num_col = num_col
        self.activity_col_write = activity_col_write
        self.num_write_cell_per_op = num_write_cell_per_op
        self.width_inv_n = constant.MIN_NMOS_SIZE * self.tech['feature_size']
        self.width_inv_p = self.tech['pn_size_ratio'] * self.width_inv_n
        self.initialized = True

    def calculate_area(self, new_height=0, new_width=0, option="NONE"):
        h_inv, w_inv = logicGate.calculate_logicgate_area(gateType = constant.INV, num_Input = 1, width_NMOS = self.width_inv_n, width_PMOS,height_Transistor_Region, tech, height_out_y, width_out_x)
        h_inv, w_inv = logicGate.calculate_logicgate_area("INV", 1, self.width_inv_n, self.width_inv_p, self.tech['feature_size'] * MAX_TRANSISTOR_HEIGHT, self.tech)
        h_unit = h_inv * 3
        w_unit = w_inv
        if new_width and option == "NONE":
            num_unit_per_row = int(new_width / w_unit)
            num_unit_per_row = min(num_unit_per_row, self.num_col)
            num_row_unit = math.ceil(self.num_col / num_unit_per_row)
            self.width = new_width
            self.height = num_row_unit * h_unit
        else:
            self.width = self.num_col * w_unit
            self.height = h_unit
        self.area = self.width * self.height
        self.cap_inv_input, self.cap_inv_output = CalculateGateCapacitance("INV", 1, self.width_inv_n, self.width_inv_p, h_inv, self.tech)

    def calculate_latency(self, ramp_input, cap_load, res_load, num_write):
        res_pull_up = CalculateOnResistance(self.width_inv_p, "PMOS", self.input_param['temperature'], self.tech)
        tr = res_pull_up * (self.cap_inv_output + self.cap_inv_input)
        gm = CalculateTransconductance(self.width_inv_p, "PMOS", self.tech)
        beta = 1 / (res_pull_up * gm)
        delay1 = horowitz(tr, beta, ramp_input)

        res_pull_down = CalculateOnResistance(self.width_inv_n, "NMOS", self.input_param['temperature'], self.tech)
        tr2 = res_pull_down * (cap_load + self.cap_inv_output) + res_load * cap_load / 2
        gm2 = CalculateTransconductance(self.width_inv_n, "NMOS", self.tech)
        beta2 = 1 / (res_pull_down * gm2)
        delay2 = horowitz(tr2, beta2, ramp_input)

        self.write_latency = (delay1 + delay2) * num_write

    def calculate_power(self, num_write):
        leakage = CalculateGateLeakage("INV", 1, self.width_inv_n, self.width_inv_p, self.input_param['temperature'], self.tech)
        self.leakage_power = leakage * self.tech['vdd'] * 3 * self.num_col
        num_active = min(self.num_write_cell_per_op, self.num_col * self.activity_col_write)
        self.write_dynamic_energy = (self.cap_inv_input + self.cap_inv_output) * self.tech['vdd']**2 * num_active * num_write

    def print_property(self):
        print("SRAM Write Driver Properties:")
        print(f"  Area: {self.area:.3e} m^2")
        print(f"  Latency: {self.write_latency:.3e} s")
        print(f"  Leakage Power: {self.leakage_power:.3e} W")
        print(f"  Write Energy: {self.write_dynamic_energy:.3e} J")
